In [ ]:
# dependencies import
from common_dependencies import *
from typing import NamedTuple
#import model_versions as mv
import logging
logger = logging.getLogger(f'main.ae_model_test')
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(290)

In [ ]:
# train params that can be easily changed
MAX_VAL = 1000
PATH_TO_MODEL = 'networks/AE'
XSHIFT = 200

MODEL_VER = 5
MODEL_NUM = 1

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER:02}n{MODEL_NUM:02}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break

dataset_desc = {'run_1': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT),),
                'run_2': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT),)}

In [ ]:
#входные и выходные данные
# reading
dataset = {'run_1':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['run_1'])])),
           'run_2':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['run_2'])]))}

# squueze datasets
for dataset_part_name, dataset_part in dataset.items():
    for data_part_name, data_part in dataset_part.items():
        if data_part_name == 'x':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1,64)
        if data_part_name == 'y':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1)

# displaying
logger.debug('\nDataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}, max={np.max(data_part)}, min={np.min(data_part)}')

In [ ]:
# Загрузка модели
ae = keras.models.load_model(PATH_TO_MODEL)
print(ae.summary())
tf.keras.utils.plot_model(
    ae,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)

In [ ]:
for dataset_part_name, dataset_part in dataset.items():
    dataset[dataset_part_name]['autoencode_result'] = ae.predict(dataset[dataset_part_name]['x'], verbose=0)

In [ ]:
for dataset_part_name, dataset_part in dataset.items():
    dataset[dataset_part_name]['all_losses'] = np.array([float(keras.losses.MeanSquaredError(reduction='sum')(true_res, pred_res)) for 
                                                true_res, pred_res in zip(dataset[dataset_part_name]['x'], ae.predict(dataset[dataset_part_name]['x'], verbose=0))])

In [ ]:
print(dataset['run_1']['all_losses'].shape)
print(dataset['run_2']['all_losses'].shape)

In [ ]:
for dataset_part_name, dataset_part in dataset.items():
    FONT_SIZE=15
    
    fig, axes = plt.subplots(1,3)
    fig.suptitle(f'Гистограммы распределение количества наблюдений для разных диапазонов значений функции потерь для {dataset_part_name}', fontsize=FONT_SIZE)
    fig.set_figwidth(22)
    fig.set_figheight(8)
    
    axes[0].hist(dataset_part['all_losses'])
    axes[0].set_title('Для всех наблюдений в выборке',fontsize = FONT_SIZE)
    axes[1].hist(dataset_part['all_losses'][dataset_part['bin']!=0])
    axes[1].set_title('Для дефектных наблюдений в выборке',fontsize = FONT_SIZE)
    axes[2].hist(dataset_part['all_losses'][dataset_part['bin']==0])
    axes[2].set_title('Для не дефектных наблюдений в выборке',fontsize = FONT_SIZE)
    
    for ax in axes:
        ax.set_xlabel('значение mean squarred error',fontsize = FONT_SIZE)
        ax.set_ylabel('кол-во элементов',fontsize = FONT_SIZE)
        ax.grid(True)
        ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
        ax.margins(0.05)
        ax.legend(fontsize = FONT_SIZE)
        
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.1)
    plt.show()

In [ ]:
# example of autoencode nondef graphs
FONT_SIZE = 20
for dataset_part_name, dataset_part in dataset.items():
    for i in range(5):
        fig, ax = plt.subplots()
        fig.set_figwidth(18)
        fig.set_figheight(8)
    
        ax.plot(dataset_part['x'][dataset_part['bin']!=0][i,1:32], dataset_part['x'][dataset_part['bin']!=0][i,32+1:], c='blue', label='original data')
        ax.plot(dataset_part['autoencode_result'][dataset_part['bin']!=0][i,1:32], dataset_part['autoencode_result'][dataset_part['bin']!=0][i,32+1:], c='red', label='predict data')
        
        ax.set_title(f'Defect graph for {dataset_part_name}',fontsize=FONT_SIZE)
        ax.legend(fontsize = FONT_SIZE)
        ax.set_xlabel('Time',fontsize=FONT_SIZE)
        ax.set_ylabel('Amplitude',fontsize=FONT_SIZE)
        ax.tick_params(axis='both', labelsize = FONT_SIZE)
        ax.grid(True)
        plt.show()
        
    for i in range(5):
        fig, ax = plt.subplots()
        fig.set_figwidth(18)
        fig.set_figheight(8)
    
        ax.plot(dataset_part['x'][dataset_part['bin']==0][i,1:32], dataset_part['x'][dataset_part['bin']==0][i,32+1:], c='blue', label='original data')
        ax.plot(dataset_part['autoencode_result'][dataset_part['bin']==0][i,1:32], dataset_part['autoencode_result'][dataset_part['bin']==0][i,32+1:], c='red', label='predict data')
        
        ax.set_title(f'Non defect graph for {dataset_part_name}',fontsize=FONT_SIZE)
        ax.legend(fontsize = FONT_SIZE)
        ax.set_xlabel('Time',fontsize=FONT_SIZE)
        ax.set_ylabel('Amplitude',fontsize=FONT_SIZE)
        ax.tick_params(axis='both', labelsize = FONT_SIZE)
        ax.grid(True)
        plt.show()

In [ ]:
# Предполагая, что у вас есть обученная модель ae
'''enc_model = keras.Model(inputs=ae.input, outputs=ae.get_layer('enc_output').output)  # Получение модели для вывода слоя enc_output

for dataset_part_name, dataset_part in dataset.items():
    dataset[dataset_part_name]['encode_result'] = enc_model.predict(dataset[dataset_part_name]['x'])'''

In [ ]:
'''FONT_SIZE = 20
name = f"""Отображения признаков из 64-x мерного пространства на двух мерное"""

fig, axes = plt.subplots(1,2)
fig.suptitle(name, fontsize=FONT_SIZE)
fig.set_figwidth(22)
fig.set_figheight(8)


axes[0].set_title('Результат для run_1', fontsize=FONT_SIZE)
axes[0].scatter(dataset['run_1']['encode_result'][dataset['run_1']['bin']==0,0], dataset['run_1']['encode_result'][dataset['run_1']['bin']==0,1], c='blue', s=3, label='non defect')
axes[0].scatter(dataset['run_1']['encode_result'][dataset['run_1']['bin']!=0,0], dataset['run_1']['encode_result'][dataset['run_1']['bin']!=0,1], c='red', s=3, label='defect')

axes[1].set_title('Результат для run_2', fontsize=FONT_SIZE)
axes[1].scatter(dataset['run_2']['encode_result'][dataset['run_2']['bin']==0,0], dataset['run_2']['encode_result'][dataset['run_2']['bin']==0,1], c='blue', s=3, label='non defect')
axes[1].scatter(dataset['run_2']['encode_result'][dataset['run_2']['bin']!=0,0], dataset['run_2']['encode_result'][dataset['run_2']['bin']!=0,1], c='red', s=3, label='defect')

for ax in axes:
    ax.set_xlabel('X',fontsize = FONT_SIZE)
    ax.set_ylabel('Y',fontsize = FONT_SIZE)
    ax.grid(True)
    ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    ax.margins(0.05)
    ax.legend(fontsize = FONT_SIZE)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.2)

#if not os.path.exists(PATH_SAVE_RESULT):
#    os.makedirs(PATH_SAVE_RESULT)
    
#plt.savefig(f'{PATH_SAVE_RESULT}/{name}.jpeg', bbox_inches='tight')
plt.show()'''